# Energy storage LCA
Desing Harald, Empa - Swiss Federal Laboratories for Material Science and Technology, St.Gallen, Switzerland
harald.desing@empa.ch

LCA of different energy storage options
- calculating life cycle embodied energy per storage capacity
- resource demand (metals, ...)
- turnaround efficiency

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import brightway2 as bw
from brightway2 import *

In [5]:
bw.projects.set_current('Energy_storage_LCA')  # creates and sets a project

Setting up databases

In [7]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [8]:
ei37_path = '<<<yourpath>>>/ecoinvent_3.7.1_cutoff_ecoSpold02/datasets'  
ei37_name = "ecoinvent 3.7.1 cutoff"  
if ei37_name in bw.databases:
    print(ei37_name + " database already present!!! No import is needed")
else:
    ei37 = bw.SingleOutputEcospold2Importer(ei37_path, ei37_name)
    ei37.apply_strategies()
    ei37.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei37.statistics()

ecoinvent 3.7.1 cutoff database already present!!! No import is needed


In [9]:
if ei37_name in bw.databases:
    print(ei37_name + " database already present!!! No import is needed")
else:
    ei37.write_database()
    ei37 = None

ecoinvent 3.7.1 cutoff database already present!!! No import is needed


In [10]:
bio = bw.Database("biosphere3")
ei = bw.Database("ecoinvent 3.7.1 cutoff")

## Methods
Chosing methods for impact assessment

In [12]:
CED = [method for method in bw.methods if "cumulative energy demand" in str(method) and 'total' not in str(method)]
#CED

In [13]:
len(CED)

8

Creating new methods for CED and CED_el

In [14]:
bw.Method(('cumulative energy demand', 'total', 'energy resources, total')).register()
CED_total = bw.Method(('cumulative energy demand', 'total', 'energy resources, total'))
print(CED_total)
CED_total.metadata['unit'] = 'MJ-eq'
CED_total.metadata['description'] = 'Total of all cumulative energy demand methods'
CED_total.metadata

Brightway2 Method: cumulative energy demand: total: energy resources, total


{'abbreviation': 'cumulative-energy-demandte.d20dc835ba5058abe813aff100f031ea',
 'unit': 'MJ-eq',
 'description': 'Total of all cumulative energy demand methods',
 'num_cfs': 13}

Creating a list of characterisation factors from all cumulative energy demand methods

In [15]:
cfs = []
for m in bw.methods:
    if "cumulative energy demand" in str(m) and 'total' not in str(m):
        method = bw.Method(m)
        for flow, cf in method.load():
            cfs.append((bw.get_activity(flow), cf))
len(cfs)

13

Writing these CFs to the new method

In [16]:
CED_total.write(cfs)
print(CED_total.metadata)

[(bw.get_activity(flow), cf) for flow, cf in CED_total.load()]

{'abbreviation': 'cumulative-energy-demandte.d20dc835ba5058abe813aff100f031ea', 'unit': 'MJ-eq', 'description': 'Total of all cumulative energy demand methods', 'num_cfs': 13}


[('Energy, gross calorific value, in biomass' (megajoule, None, ('natural resource', 'biotic')),
  1.0),
 ('Coal, brown, in ground' (kilogram, None, ('natural resource', 'in ground')),
  9.9),
 ('Coal, hard, unspecified, in ground' (kilogram, None, ('natural resource', 'in ground')),
  19.1),
 ('Gas, mine, off-gas, process, coal mining' (cubic meter, None, ('natural resource', 'in ground')),
  39.8),
 ('Gas, natural, in ground' (cubic meter, None, ('natural resource', 'in ground')),
  38.293),
 ('Oil, crude, in ground' (kilogram, None, ('natural resource', 'in ground')),
  45.8),
 ('Peat, in ground' (kilogram, None, ('natural resource', 'biotic')), 9.9),
 ('Energy, geothermal, converted' (megajoule, None, ('natural resource', 'in ground')),
  1.0),
 ('Uranium, in ground' (kilogram, None, ('natural resource', 'in ground')),
  560000.0),
 ('Energy, gross calorific value, in biomass, primary forest' (megajoule, None, ('natural resource', 'biotic')),
  1.0),
 ('Energy, solar, converted' (m

Cumulative energy demand for electric energy equivalents

In [17]:
conversion_factors_el = [
    {
        'category':'wind',
        'efficiency': 1
    },
    {
        'category':'biomass',
        'efficiency': 0.25
    },
    {
        'category':'fossil',
        'efficiency': 0.36
    },
    {
        'category':'geothermal',
        'efficiency': 1
    },
    {
        'category':'nuclear',
        'efficiency': 0.33
    },
    {
        'category':'primary forest',
        'efficiency': 0.25
    },
    {
        'category':'solar',
        'efficiency': 1
    },
    {
        'category':'water',
        'efficiency': 1
    },
    
]

In [18]:
bw.Method(('cumulative energy demand, electricity', 'total', 'energy resources, total')).register()

CED_el_total = bw.Method(('cumulative energy demand, electricity', 'total', 'energy resources, total'))
print(CED_el_total)

CED_el_total.metadata['unit'] = 'MJ-eq'
CED_el_total.metadata['description'] = 'Total of all cumulative energy demand methods converted to electric energy equivalents'
CED_el_total.metadata

Brightway2 Method: cumulative energy demand, electricity: total: energy resources, total


{'abbreviation': 'cumulative-energy-demand-electricityte.ef9e1bb3a0894bf337d2c25c2f2b3602',
 'unit': 'MJ-eq',
 'description': 'Total of all cumulative energy demand methods converted to electric energy equivalents',
 'num_cfs': 13}

In [19]:
cfs_el = []
for m in bw.methods:
    if "cumulative energy demand" in str(m) and 'total' not in str(m) and 'electricity' not in str(m):
        method = bw.Method(m)
        for conv_f in conversion_factors_el:
            if conv_f['category'] in str(method):
                for flow, cf in method.load():
                    cfs_el.append((bw.get_activity(flow), cf * conv_f['efficiency']))
len(cfs_el)

13

In [20]:
CED_el_total.write(cfs_el)
print(CED_el_total.metadata)

[(bw.get_activity(flow), cf) for flow, cf in CED_el_total.load()]

{'abbreviation': 'cumulative-energy-demand-electricityte.ef9e1bb3a0894bf337d2c25c2f2b3602', 'unit': 'MJ-eq', 'description': 'Total of all cumulative energy demand methods converted to electric energy equivalents', 'num_cfs': 13}


[('Energy, gross calorific value, in biomass' (megajoule, None, ('natural resource', 'biotic')),
  0.25),
 ('Coal, brown, in ground' (kilogram, None, ('natural resource', 'in ground')),
  3.564),
 ('Coal, hard, unspecified, in ground' (kilogram, None, ('natural resource', 'in ground')),
  6.876),
 ('Gas, mine, off-gas, process, coal mining' (cubic meter, None, ('natural resource', 'in ground')),
  14.327999999999998),
 ('Gas, natural, in ground' (cubic meter, None, ('natural resource', 'in ground')),
  13.78548),
 ('Oil, crude, in ground' (kilogram, None, ('natural resource', 'in ground')),
  16.488),
 ('Peat, in ground' (kilogram, None, ('natural resource', 'biotic')), 3.564),
 ('Energy, geothermal, converted' (megajoule, None, ('natural resource', 'in ground')),
  1.0),
 ('Uranium, in ground' (kilogram, None, ('natural resource', 'in ground')),
  184800.0),
 ('Energy, gross calorific value, in biomass, primary forest' (megajoule, None, ('natural resource', 'biotic')),
  0.25),
 ('Ene

For checking, if the methods are created:

In [21]:
[m for m in bw.methods if "cumulative energy demand" in str(m)]

[('cumulative energy demand',
  'biomass',
  'renewable energy resources, biomass'),
 ('cumulative energy demand',
  'fossil',
  'non-renewable energy resources, fossil'),
 ('cumulative energy demand',
  'geothermal',
  'renewable energy resources, geothermal, converted'),
 ('cumulative energy demand',
  'nuclear',
  'non-renewable energy resources, nuclear'),
 ('cumulative energy demand',
  'primary forest',
  'non-renewable energy resources, primary forest'),
 ('cumulative energy demand',
  'solar',
  'renewable energy resources, solar, converted'),
 ('cumulative energy demand',
  'water',
  'renewable energy resources, potential (in barrage water), converted'),
 ('cumulative energy demand',
  'wind',
  'renewable energy resources, kinetic (in wind), converted'),
 ('cumulative energy demand', 'total', 'energy resources, total'),
 ('cumulative energy demand, electricity', 'total', 'energy resources, total')]

In [22]:
CED_method_key = [m for m in bw.methods if "cumulative energy demand" in str(m) and 'total' in str(m)]

Resource consumption LCIA

In [23]:
resource_consumption = [m for m in bw.methods if "resource consumption" in str(m) and "w/o LT" not in str(m)]
#resource_consumption

In [24]:
len(resource_consumption)

25

Make a combined list of all methods

In [25]:
LCIA_methods = [m for m in bw.methods if "cumulative energy demand" in str(m) 
                or "resource consumption" in str(m) and "w/o LT" not in str(m)
                or "climate change total" in str(m) and "ILCD 2.0" in str(m) and 'no LT' not in str(m)
               ]
#LCIA_methods.append(resource_consumption)
len(LCIA_methods)
#LCIA_methods

36

## Creating new datasets for analysis
### Hydro pumped storage
combining all hydro pumped datasets from ecoinvent into a global mix

In [26]:
pumped_storage_datasets = [a for a in ei if 'pumped storage' in a['name']]
n_pumped_storage_datasets = len(pumped_storage_datasets)
n_pumped_storage_datasets

69

Get keys for all datasets in ecoinvent for hydro, pumped storage

In [27]:
PHS_keys = [a.key[1] for a in ei if 'pumped storage' in a['name']]

Create new database with one global dataset without exchanges

In [28]:
pumped_hydro = Database("pumped hydro storage")

In [29]:
pumped_hydro.write({    
    ("pumped hydro storage", "1"):{
        "name" : "electricity production, hydro, pumped storage",
        "exchanges": [],
        'reference product': 'electricity, high voltage',
        'production amount': 1.0,
        'unit': 'kilowatt hour',
        'location': 'GLO',
    },
   
    ("pumped hydro storage", "2"):{
        "name" : "electricity production, hydro, pumped storage, embodied",
        "exchanges": [],
        'reference product': 'electricity, high voltage',
        'production amount': 1.0,
        'unit': 'kilowatt hour',
        'location': 'GLO',
    }
})

Writing activities to SQLite3 database:
0% [##] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/01/2021 08:10:22
  Finished: 06/01/2021 08:10:22
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.45


In [31]:
pumped_storage = pumped_hydro.get("1")
pumped_storage.as_dict()

{'name': 'electricity production, hydro, pumped storage',
 'reference product': 'electricity, high voltage',
 'production amount': 1.0,
 'unit': 'kilowatt hour',
 'location': 'GLO',
 'database': 'pumped hydro storage',
 'code': '1'}

Add exchanges to datasets for pumped hydro in ecoinvent; based on average for all datasets (not by storage capacity)

In [32]:
for key in PHS_keys:
    ds = ei.get(key)
    new_exc = pumped_storage.new_exchange(input=ds, amount=1/n_pumped_storage_datasets, type='technosphere')
    new_exc.save()

In [33]:
for exc in pumped_storage.exchanges():
    print(exc)

Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, ES, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, GLO, None)>
Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, IT, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, GLO, None)>
Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, SI, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, GLO, None)>
Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, BA, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, GLO, None)>
Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, US-WECC, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, GLO, None)>
Excha

Turnaround efficiency of pumped hydro storage

looking for exchanges that are input of electricity

In [34]:
PHS_key = [a.key[1] for a in ei if 'pumped storage' in a['name'] and 'LU' in a['location']]
ds = ei.get(PHS_key)
#ds = ei.get(PHS_keys[0])
input_electricity = [exc for exc in ds.exchanges() if 'electricity, high voltage' in exc['name'] and exc['input'][1]!=exc['output'][1]]#[0]
input_electricity#.as_dict()

[Exchange: 0.991 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, DE, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, LU, None)>,
 Exchange: 0.439 kilowatt hour 'market for electricity, high voltage' (kilowatt hour, BE, None) to 'electricity production, hydro, pumped storage' (kilowatt hour, LU, None)>]

adding up the different market datasets of inputs:

In [35]:
input_electricity_amount = 0
for exc in ds.exchanges():
    if 'electricity, high voltage' in exc['name'] and exc['input'][1]!=exc['output'][1]:
        input_electricity_amount += exc['amount']
input_electricity_amount

1.43

Calculating turnaround efficiency for each dataset and store it in a dictionary:

In [36]:
PHS_turnaround_efficiency = []
for key in PHS_keys:
    ds = ei.get(key)
    input_electricity_amount = 0
    for exc in ds.exchanges():
        if 'electricity, high voltage' in exc['name'] and exc['input'][1]!=exc['output'][1]:
            input_electricity_amount += exc['amount']
    PHS_turnaround_efficiency.append(
        {
            'dataset': ds['name'],
            'location': ds['location'],
            'turnaround efficiency': 1/input_electricity_amount
        }
    )
PHS_turnaround_efficiency

[{'dataset': 'electricity production, hydro, pumped storage',
  'location': 'ES',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'IT',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'SI',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'BA',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'US-WECC',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'CA-AB',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': 'US-FRCC',
  'turnaround efficiency': 0.6993006993006994},
 {'dataset': 'electricity production, hydro, pumped storage',
  'location': '

Calculate the average across datasets:

In [37]:
PHS_turnaround_efficiency_average = 0
for ds in PHS_turnaround_efficiency:
    PHS_turnaround_efficiency_average += ds['turnaround efficiency'] / len(PHS_turnaround_efficiency)

PHS_turnaround_efficiency_average

0.6993006993006988

### Create hydro power datasets with only embodied energy
Remove the electricity input into the storage datasets

Copy existing datasets from ecoinvent in new database:
(hint: any changes to a dataset need to be saved, otherwise it doesn't recognise it)

In [38]:
for key in PHS_keys:
    ds = ei.get(key)
    ds_copy = ds.copy()
    ds_copy['database'] = 'pumped hydro storage'
    ds_copy['comment'] = 'dataset without input of electricity; only embodied energy'
    ds_copy.save()

Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro storage`
Successfully switch activity dataset to database `pumped hydro s

In [39]:
len(pumped_hydro)

71

removing exchanges with input of electricity

In [40]:
PHS_embodied_keys = [a.key[1] for a in pumped_hydro if 'pumped storage' in a['name'] and 'GLO' not in a['location']]
len(PHS_embodied_keys)

69

In [42]:
for key in PHS_embodied_keys:
    ds = pumped_hydro.get(key)
    for exc in ds.exchanges():
        if 'electricity, high voltage' in exc['name'] and exc['input'][1]!=exc['output'][1]:
            for input_key in exc.input.key[1]:
                exc.delete()

In [43]:
pumped_storage2 = pumped_hydro.get("2")
pumped_storage2.as_dict()

{'name': 'electricity production, hydro, pumped storage, embodied',
 'reference product': 'electricity, high voltage',
 'production amount': 1.0,
 'unit': 'kilowatt hour',
 'location': 'GLO',
 'database': 'pumped hydro storage',
 'code': '2'}

In [44]:
for key in PHS_embodied_keys:
    ds = pumped_hydro.get(key)
    new_exc = pumped_storage2.new_exchange(input=ds, amount=1/n_pumped_storage_datasets, type='technosphere')
    new_exc.save()

In [45]:
list(pumped_storage2.exchanges())

[Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, CA-NB, None) to 'electricity production, hydro, pumped storage, embodied' (kilowatt hour, GLO, None)>,
 Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, CN-HB, None) to 'electricity production, hydro, pumped storage, embodied' (kilowatt hour, GLO, None)>,
 Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, PL, None) to 'electricity production, hydro, pumped storage, embodied' (kilowatt hour, GLO, None)>,
 Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, KR, None) to 'electricity production, hydro, pumped storage, embodied' (kilowatt hour, GLO, None)>,
 Exchange: 0.014492753623188406 kilowatt hour 'electricity production, hydro, pumped storage' (kilowatt hour, CN-JS, None) to 'electricity production, hyd

In [46]:
list([pumped_storage.key, pumped_storage2.key])

[('pumped hydro storage', '1'), ('pumped hydro storage', '2')]

check if the technosphere matrix is square (an can be solved)

## Load activity data from excel
Data for synfuel production with H2 from PEM, methanisation and CO2 from atmosphere (DAC (CCU)). Datasets from Zhang et al. (2017)

Download datasets from Zhang et al. (2017) supplementary information and safe it in a directory of your choice. www.doi.org/10.1016/j.apenergy.2016.12.098
File path to input file

In [48]:
filepath_Zhang = '<<<your path>>>/Synfuel+DAC.xlsx'

Import activities

In [49]:
if 'P2M Atmosphere' in bw.databases:
    print('Database already present, no import needed')
else:
    Synfuel_ds = bw.ExcelImporter(filepath_Zhang)
    Synfuel_ds.apply_strategies()
    Synfuel_ds.match_database(fields=['name', 'unit', 'location'])
    Synfuel_ds.match_database("ecoinvent 3.7.1 cutoff",fields=['name', 'unit', 'location', 'reference product'])
    Synfuel_ds.match_database("biosphere3",fields=['name', 'unit', 'compartment'])
    Synfuel_ds.statistics()

Database already present, no import needed


In case there are unlinked exchanges, list them here:

In [50]:
if 'P2M Atmosphere' in bw.databases:
    print('Database already present, no import needed')
else:
    for exc in Synfuel_ds.unlinked:
        print(exc)

Database already present, no import needed


Still unlinked exchanges, so not writing database!

Writing the imported datasets to a database

In [52]:
if 'P2M Atmosphere' in bw.databases:
    print('Database already present, no import needed')
else:
    Synfuel_ds.write_excel()
    Synfuel_ds.write_database()

Database already present, no import needed


In [53]:
Synfuel_db = bw.Database("P2M Atmosphere")
len(Synfuel_db)

26

Select electricity production from methane, PEM, DAC

In [54]:
el_CH4_PEM_DAC_key = [ds.key[1] for ds in Synfuel_db if 'electricity production' in ds['name']]
el_CH4_PEM_DAC = Synfuel_db.get(el_CH4_PEM_DAC_key)
el_CH4_PEM_DAC

'electricity production, methane, from thermo-chemical methanation, PEM, DAC' (kilowatt hour, None, None)

calculating turnaround efficiency

In [55]:
CH4_input_amount = np.sum([exc.amount for exc in el_CH4_PEM_DAC.exchanges() if 'methane' in exc.input['name']])
CH4 = Synfuel_db.get([exc.input['code'] for exc in el_CH4_PEM_DAC.exchanges() if 'methane' in exc.input['name']])
CH4

'methane, from thermo-chemical methanation, PEM Electrolyzer, CO2 from DAC' (cubic meter, None, None)

In [56]:
CO2_input_amount = np.sum([exc.amount for exc in CH4.exchanges() if 'carbon dioxide' in exc.input['name']])
CO2 = Synfuel_db.get([exc.input['code'] for exc in CH4.exchanges() if 'carbon dioxide' in exc.input['name']])
CO2

'carbon dioxide, captured from atmosphere' (kilogram, CH, None)

In [57]:
el_input_to_CO2_amount = np.sum([exc.amount for exc in CO2.exchanges() if 'market for electricity' in exc.input['name']])
el_input_to_CO2_amount

0.37

In [58]:
H2_input_amount = np.sum([exc.amount for exc in CH4.exchanges() if 'hydrogen, gaseous' in exc.input['name']])
H2 = Synfuel_db.get([exc.input['code'] for exc in CH4.exchanges() if 'hydrogen, gaseous' in exc.input['name']])
H2

'hydrogen, gaseous, from PEM electrolysis, 30 bars' (cubic meter, None, None)

In [59]:
el_input_to_H2_amount = np.sum([exc.amount for exc in H2.exchanges() if 'market' and 'for electricity' in exc.input['name'] or 'electricity production' in exc.input['name']])
el_input_to_H2_amount

4.908353615549993

In [60]:
el_input_CH4 = CH4_input_amount * (CO2_input_amount * el_input_to_CO2_amount + H2_input_amount * el_input_to_H2_amount)
el_input_CH4

6.085393448896195

In [61]:
Synfuel_CH4_PEM_DAC_turnaround_efficiency = 1/el_input_CH4
Synfuel_CH4_PEM_DAC_turnaround_efficiency

0.16432791213876008

Create an inventory with only embodied energy by deleting all exchanges for operational energy in all datasets in the excel file.

Importing the same database without operational electricity (i.e. only with embodied and auxiliary energy)

In [62]:
filepath_synfuel_embodied = '<<<your path>>>/Synfuel+DAC_embodied.xlsx'

In [63]:
if 'P2M Atmosphere embodied' in bw.databases:
    print('Database already present, no import needed')
else:
    Synfuel_embodied_ds = bw.ExcelImporter(filepath_synfuel_embodied)
    Synfuel_embodied_ds.apply_strategies()
    Synfuel_embodied_ds.match_database(fields=['name', 'unit', 'location'])
    Synfuel_embodied_ds.match_database("ecoinvent 3.7.1 cutoff",fields=['name', 'unit', 'location', 'reference product'])
    Synfuel_embodied_ds.match_database("biosphere3",fields=['name', 'unit', 'compartment'])
    Synfuel_embodied_ds.statistics()

Database already present, no import needed


In case there are unlinked exchanges, list them here:

In [64]:
if 'P2M Atmosphere embodied' in bw.databases:
    print('Database already present, no import needed')
else:
    for exc in Synfuel_embodied_ds.unlinked:
        print(exc)

Database already present, no import needed


Still unlinked exchanges, so not writing database!

Writing the imported datasets to a database

In [67]:
if 'P2M Atmosphere embodied' in bw.databases:
    print('Database already present, no import needed')
else:
    Synfuel_embodied_ds.write_excel()
    Synfuel_embodied_ds.write_database()

Database already present, no import needed


In [68]:
Synfuel_embodied_db = bw.Database("P2M Atmosphere embodied")
len(Synfuel_embodied_db)

26

Select electricity production from methane, PEM, DAC

In [69]:
el_CH4_PEM_DAC_embodied_key = [ds.key[1] for ds in Synfuel_embodied_db if 'electricity production' in ds['name']]
#el_CH4_PEM_DAC_embodied_key
el_CH4_PEM_DAC_embodied = Synfuel_embodied_db.get(el_CH4_PEM_DAC_embodied_key)
el_CH4_PEM_DAC_embodied

'electricity production, methane, from thermo-chemical methanation, PEM, DAC' (kilowatt hour, None, None)

### Battery datasets (Crenna et al. 2021)
Loading datasets from Crenna et al. 2021 (now alos contained in ecoinvent v3.8). If you donÄt have ecoinvent 3.8, an excel inventory file needs to be created in the same structure as Zhang et al. www.doi.org/10.1016/j.resconrec.2021.105619

In [72]:
filepath_battery = '<<<your path>>>/Battery_db.xlsx'

In [73]:
if 'Empa Battery Database' in bw.databases:
    print('Database already present, no import needed')
else:
    Battery_db = bw.ExcelImporter(filepath_battery)
    Battery_db.apply_strategies()
    Battery_db.match_database(fields=['name', 'unit', 'location'])
    Battery_db.match_database("ecoinvent 3.7.1 cutoff",fields=['name', 'unit', 'location', 'reference product'])
    Battery_db.match_database("biosphere3",fields=['name', 'unit', 'compartment'])
    Battery_db.statistics()

Extracted 4 worksheets in 0.15 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.33 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
31 datasets
349 exchanges
0 unlinked exchang

In [74]:
if 'Empa Battery Database' in bw.databases:
    print('Database already present, no import needed')
else:
    for exc in list(Battery_db.unlinked):
        print(exc)

In [75]:
#list(Battery_db.unlinked)

Write database

In [76]:
if 'Empa Battery Database' in bw.databases:
    print('Database already present, no import needed')
else:
    Battery_db.write_excel()
    Battery_db.write_database()

Writing activities to SQLite3 database:


Wrote matching file to:
C:\Users\dha\AppData\Local\pylca\Brightway3\Energy_storage_LCA.973a4c974012c913c92ae61bd43e2cf6\output\db-matching-Empa-Battery-Database.xlsx


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/01/2021 08:11:09
  Finished: 06/01/2021 08:11:09
  Total time elapsed: 00:00:00
  CPU %: 71.00
  Memory %: 1.63
Created database: Empa Battery Database


In [77]:
Battery_db = bw.Database("Empa Battery Database")
len(Battery_db)

31

In [78]:
NMC111_key = [act.key[1] for act in Battery_db if 'battery, Li-ion, NMC111' in act['name']][0] 
NMC111 = Battery_db.get(NMC111_key)
NMC111

'battery, Li-ion, NMC111, rechargeable, prismatic' (kilogram, GLO, None)

In [79]:
NMC811_key = [act.key[1] for act in Battery_db if 'battery, Li-ion, NMC811' in act['name']][0] 
NMC811 = Battery_db.get(NMC811_key)
NMC811

'battery, Li-ion, NMC811, rechargeable, prismatic' (kilogram, GLO, None)

In [80]:
NCA_key = [act.key[1] for act in Battery_db if 'battery, Li-ion, NCA' in act['name']][0] 
NCA = Battery_db.get(NCA_key)
NCA

'battery, Li-ion, NCA, rechargeable, prismatic' (kilogram, GLO, None)

In [81]:
Co_key = [act.key[1] for act in Battery_db if 'cobalt hydroxide' in act['name']][0] 
Co = Battery_db.get(Co_key)
Co

'cobalt hydroxide' (kilogram, CO, None)

## Performing LCA:
using a calculation setup and multi LCA

Functional unit: "storage of 1 kWh output in electric energy"

List of alternatives to compare. Functional unit is always 1.
- pumped hydro
- compresed air
- rail energy storage
- batteries

In [82]:
list_functional_units = [{pumped_storage.key:1}, {pumped_storage2.key:1}, {el_CH4_PEM_DAC.key:1}, {el_CH4_PEM_DAC_embodied.key:1}]
#list_methods = [GWP.name, ADP.name, biodiv.name, UBP.name]
list_methods = LCIA_methods # or CED or resource consumption
list_functional_units

[{('pumped hydro storage', '1'): 1},
 {('pumped hydro storage', '2'): 1},
 {('P2M Atmosphere', '055a72261a723d34e5241587049c16d9'): 1},
 {('P2M Atmosphere embodied', '055a72261a723d34e5241587049c16d9'): 1}]

In [83]:
bw.calculation_setups['energy_storage_LCA'] = {'inv':list_functional_units, 'ia':list_methods}

In [84]:
Energy_storage_LCA = bw.MultiLCA('energy_storage_LCA')

In [85]:
Energy_storage_LCA.results.shape

(4, 36)

In [86]:
results = Energy_storage_LCA.results

In [87]:
index_CED = np.zeros(1, dtype=int)
m_i=0
i=0
for m in Energy_storage_LCA.methods:
    if 'cumulative energy demand' in str(m) and 'total' in str(m) and 'electricity' not in str(m):
        index_CED[i] = m_i
        i+=1
    m_i+=1

index_CED

array([34])

In [88]:
storage_CED_total = results[:,index_CED]
storage_CED_total

array([[15.7233211 ],
       [ 0.0580086 ],
       [31.86709388],
       [ 0.52075458]])

In [89]:
index_CED_el = np.zeros(1, dtype=int)
m_i=0
i=0
for m in Energy_storage_LCA.methods:
    if 'cumulative energy demand' in str(m) and 'total' in str(m) and 'electricity' in str(m):
        index_CED_el[i] = m_i
        i+=1
    m_i+=1

index_CED_el

array([35])

In [90]:
storage_CED_el_total = results[:,index_CED_el]
storage_CED_el_total

array([[6.53726625e+00],
       [2.28527366e-02],
       [2.68345140e+01],
       [2.23989356e-01]])

converting the results into embodied energy per stored energy (i.e. the functional unit of 1 kWh); therefore converting CED_el (which is in MJ) into kWh (1 MJ = 1/3.6 kWh) 

In [91]:
embodied_energy_per_stored_energy = storage_CED_el_total / 3.6
embodied_energy_per_stored_energy

array([[1.81590729e+00],
       [6.34798238e-03],
       [7.45403166e+00],
       [6.22192655e-02]])

Display of results

In [92]:
scores = pd.DataFrame(Energy_storage_LCA.results, columns=Energy_storage_LCA.methods)
as_activities = [
    (bw.get_activity(key), amount) 
    for dct in Energy_storage_LCA.func_units 
    for key, amount in dct.items()
]
nicer_fu = pd.DataFrame(
    [
        (x['database'], x['code'], x['name'], x['unit'], y) 
        for x, y in as_activities
    ], 
    columns=('Database', 'Code', 'Name', 'Unit', 'Amount')
)
pd.concat([nicer_fu, scores], axis=1).T

,0,1,2,3
Database,pumped hydro storage,pumped hydro storage,P2M Atmosphere,P2M Atmosphere embodied
Code,1,2,055a72261a723d34e5241587049c16d9,055a72261a723d34e5241587049c16d9
Name,"electricity production, hydro, pumped storage","electricity production, hydro, pumped storage,...","electricity production, methane, from thermo-c...","electricity production, methane, from thermo-c..."
Unit,kilowatt hour,kilowatt hour,kilowatt hour,kilowatt hour
Amount,1,1,1,1
"(EDIP (obsolete), resource consumption, non-renewable resources, antimony)",1.62343e-11,7.86662e-13,2.0151e-09,1.21519e-11
"(EDIP (obsolete), resource consumption, non-renewable resources, cadmium)",1.53875e-09,1.59023e-10,5.75206e-08,6.4122e-10
"(EDIP (obsolete), resource consumption, non-renewable resources, cerium)",1.05249e-08,4.00645e-10,4.8112e-07,2.84903e-08
"(EDIP (obsolete), resource consumption, non-renewable resources, copper)",3.11366e-05,1.22891e-06,0.00220803,7.16185e-05
"(EDIP (obsolete), resource consumption, non-renewable resources, molybdenum)",0,0,0,0


# Performing LCA2: for battery

Functional unit is storage capacity

NMC111: 0.143 kWh/kg

NMC811: 0.149 kWh/kg

NCA: 0.159 kWh/kg

In [95]:
list_functional_units2 = [{NMC111.key:1/0.143},{NMC811.key:1/0.149},{NCA.key:1/0.159},{NMC111.key:1},{NMC811.key:1},{NCA.key:1}]
list_functional_units2

[{('Empa Battery Database',
   '418c1f5539e61f1f7a79f81295efcf59'): 6.993006993006993},
 {('Empa Battery Database',
   'ca52eed1ad12b485efd2d2d1ce6044c9'): 6.7114093959731544},
 {('Empa Battery Database',
   '041b991f89f99a68f54d69e88a7eb38f'): 6.289308176100628},
 {('Empa Battery Database', '418c1f5539e61f1f7a79f81295efcf59'): 1},
 {('Empa Battery Database', 'ca52eed1ad12b485efd2d2d1ce6044c9'): 1},
 {('Empa Battery Database', '041b991f89f99a68f54d69e88a7eb38f'): 1}]

In [96]:
bw.calculation_setups['battery_storage_LCA'] = {'inv':list_functional_units2, 'ia':list_methods}

In [97]:
battery_storage_LCA = bw.MultiLCA('battery_storage_LCA')

In [98]:
battery_storage_LCA.results.shape

(6, 36)

In [99]:
battery_results = battery_storage_LCA.results

In [100]:
scores2 = pd.DataFrame(battery_storage_LCA.results, columns=battery_storage_LCA.methods )
as_activities = [
    (bw.get_activity(key), amount) 
    for dct in battery_storage_LCA.func_units 
    for key, amount in dct.items()
]
nicer_fu = pd.DataFrame(
    [
        (x['database'], x['code'], x['name'], x['unit'], y) 
        for x, y in as_activities
    ], 
    columns=('Database', 'Code', 'Name', 'Unit', 'Amount')
)
pd.concat([nicer_fu, scores2], axis=1).T

,0,1,2,3,4,5
Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database
Code,418c1f5539e61f1f7a79f81295efcf59,ca52eed1ad12b485efd2d2d1ce6044c9,041b991f89f99a68f54d69e88a7eb38f,418c1f5539e61f1f7a79f81295efcf59,ca52eed1ad12b485efd2d2d1ce6044c9,041b991f89f99a68f54d69e88a7eb38f
Name,"battery, Li-ion, NMC111, rechargeable, prismatic","battery, Li-ion, NMC811, rechargeable, prismatic","battery, Li-ion, NCA, rechargeable, prismatic","battery, Li-ion, NMC111, rechargeable, prismatic","battery, Li-ion, NMC811, rechargeable, prismatic","battery, Li-ion, NCA, rechargeable, prismatic"
Unit,kilogram,kilogram,kilogram,kilogram,kilogram,kilogram
Amount,6.99301,6.71141,6.28931,1,1,1
"(EDIP (obsolete), resource consumption, non-renewable resources, antimony)",4.75123e-07,4.18148e-07,4.23441e-07,6.79426e-08,6.2304e-08,6.73271e-08
"(EDIP (obsolete), resource consumption, non-renewable resources, cadmium)",1.86288e-05,1.66649e-05,1.67918e-05,2.66392e-06,2.48307e-06,2.6699e-06
"(EDIP (obsolete), resource consumption, non-renewable resources, cerium)",0.000214997,0.000264727,0.000246695,3.07446e-05,3.94443e-05,3.92245e-05
"(EDIP (obsolete), resource consumption, non-renewable resources, copper)",2.96759,2.04877,2.17363,0.424366,0.305267,0.345607
"(EDIP (obsolete), resource consumption, non-renewable resources, molybdenum)",0,0,0,0,0,0


Calculating total CED

In [102]:
battery_index_CED = np.zeros(1, dtype=int)
m_i=0
i=0
for m in battery_storage_LCA.methods:
    if 'cumulative energy demand' in str(m) and 'total' in str(m) and 'electricity' not in str(m):
        battery_index_CED[i] = m_i
        i+=1
    m_i+=1

battery_index_CED

array([34])

In [103]:
battery_CED_total = battery_results[:,battery_index_CED]
battery_CED_total

array([[2866.05131227],
       [2189.26342135],
       [2250.82993901],
       [ 409.84533765],
       [ 326.20024978],
       [ 357.8819603 ]])

In [104]:
battery_index_CED_el = np.zeros(1, dtype=int)
m_i=0
i=0
for m in battery_storage_LCA.methods:
    if 'cumulative energy demand' in str(m) and 'total' in str(m) and 'electricity' in str(m):
        battery_index_CED_el[i] = m_i
        i+=1
    m_i+=1

battery_index_CED_el

array([35])

In [105]:
battery_CED_el_total = battery_results[:,battery_index_CED_el]
battery_CED_el_total

array([[1646.62226106],
       [1163.56593853],
       [1221.51658671],
       [ 235.46698333],
       [ 173.37132484],
       [ 194.22113729]])

converting the results into embodied energy per storage capacity (i.e. the functional unit of 1 kWh of storage capacity); therefore converting CED_el (which is in MJ) into kWh (1 MJ = 1/3.6 kWh) 

In [106]:
battery_embodied_energy_per_storage_capacity = battery_CED_el_total / 3.6
battery_embodied_energy_per_storage_capacity

array([[457.39507252],
       [323.2127607 ],
       [339.31016298],
       [ 65.40749537],
       [ 48.15870134],
       [ 53.95031591]])

In [107]:
scores = pd.DataFrame(battery_storage_LCA.results[:,battery_index_CED[0]], columns= [(battery_storage_LCA.methods[battery_index_CED[0]], bw.Method(battery_storage_LCA.methods[battery_index_CED[0]]).metadata['unit'])])
scores2 = pd.DataFrame(battery_storage_LCA.results[:,battery_index_CED_el[0]], columns= [(battery_storage_LCA.methods[battery_index_CED_el[0]], bw.Method(battery_storage_LCA.methods[battery_index_CED_el[0]]).metadata['unit'])])
scores3 = pd.DataFrame(battery_embodied_energy_per_storage_capacity, columns=['embodied energy per storage capacity'])
header = pd.DataFrame(
    [
        (x['database'], x['name'], x['unit'], y) 
        for x, y in [
            (bw.get_activity(key), amount) 
            for dct in battery_storage_LCA.func_units 
            for key, amount in dct.items()
            ]
    ], 
    columns=('Database','Name', 'Unit', 'Amount')
)
pd.concat([header, scores, scores2, scores3], axis=1).T

,0,1,2,3,4,5
Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database,Empa Battery Database
Name,"battery, Li-ion, NMC111, rechargeable, prismatic","battery, Li-ion, NMC811, rechargeable, prismatic","battery, Li-ion, NCA, rechargeable, prismatic","battery, Li-ion, NMC111, rechargeable, prismatic","battery, Li-ion, NMC811, rechargeable, prismatic","battery, Li-ion, NCA, rechargeable, prismatic"
Unit,kilogram,kilogram,kilogram,kilogram,kilogram,kilogram
Amount,6.99301,6.71141,6.28931,1,1,1
"((cumulative energy demand, total, energy resources, total), MJ-eq)",2866.05,2189.26,2250.83,409.845,326.2,357.882
"((cumulative energy demand, electricity, total, energy resources, total), MJ-eq)",1646.62,1163.57,1221.52,235.467,173.371,194.221
embodied energy per storage capacity,457.395,323.213,339.31,65.4075,48.1587,53.9503
